In [1]:
##Warp()函数快速处理
from osgeo import gdal, osr

In [2]:
path_img = 'E:\开源\chenggong_tif.tif'
path_reproj= 'E:\开源\chenggong_tif_wgs_2.tif'
##reproj是在gdal.ipynb中完成utm转wgs84的图像
warp_reproj = gdal.Warp(srcDSOrSrcDSTab=path_img,\
    destNameOrDestDS= path_reproj,\
        dstSRS ='EPSG:4326')
warp_reproj = None

TypeError: in method 'wrapper_GDALWarpDestName', argument 4 of type 'GDALWarpAppOptions *'

In [3]:
path_resam = 'E:\开源\chenggong_resam.tif'
warp_resam = gdal.Warp(srcDSOrSrcDSTab=path_img,\
                       destNameOrDestDS=path_resam,\
                        xRes = 25,
                        yRes = 25,
                        resampleAlg = gdal.GRA_Bilinear
)
warp_resam = None
##查看Extent空间范围发生变化，通过warp resam，目的是为了能整除


In [4]:
##用gdal数据库完成栅格数据的裁剪
from osgeo import gdal
from numpy import np

ImportError: cannot import name 'np' from 'numpy' (D:\miniconda\lib\site-packages\numpy\__init__.py)

In [ ]:
path_img = 'E:\开源\kunming_tiff.tif'

In [4]:
dset = gdal.Open(path_img)
geo_trans = dset.GetGeoTransform()
print(geo_trans)

x_min, y_max = geo_trans[0], geo_trans[3]
x_res, y_res = geo_trans[1], geo_trans[5]

x_max = x_min+dset.RasterXSize*x_res
y_min = y_max+dset.RasterYSize*y_res
print('extent:',x_min, x_max, y_min, y_max)
##第1 4为角点，2 6空间分辨率，3 5旋转系数

(874420.0, 10.0, 0.0, 2769600.0, 0.0, -10.0)
extent: 874420.0 904220.0 2736990.0 2769600.0


In [6]:
x_min_subs, x_max_subs = 887957, 890779
y_min_subs, y_max_subs = 2750586, 2752375


In [36]:
col_start_subs = (x_min_subs-x_min)/x_res
row_start_subs = (y_max_subs-y_max)/y_res
print(col_start_subs, row_start_subs)
##为什么会有小数呢？请看video

1353.7 1722.5


In [33]:
col_start_subs_update, row_start_subs_update = round(col_start_subs), round(row_start_subs)
print(col_start_subs_update, row_start_subs_update)


1354 1722


In [37]:
##左上角：
x_min_subs_update = x_min + col_start_subs_update*x_res
y_max_subs_update = y_max + row_start_subs_update*y_res
print(x_min_subs_update, y_max_subs_update)

887960.0 2752380.0


In [38]:
x_size_subs = (x_max_subs - x_min_subs_update)/x_res
y_size_subs = (y_max_subs_update-y_min_subs)/y_res
print(x_size_subs, y_size_subs)
##裁剪后的尺寸为小数不合理

281.9 -179.4


In [39]:
x_size_subs_update, y_size_subs_update = round(x_size_subs), round(y_size_subs)
print(x_size_subs_update, y_size_subs_update)

282 -179


In [40]:
x_max_subs_update = x_min_subs + x_size_subs_update*x_res
##右下角y
y_min_subs_update = y_max_subs_update + y_size_subs_update*y_res
print(x_max_subs_update, y_min_subs_update)

890777.0 2754170.0


In [41]:
extent_subs_update = [x_min_subs_update, x_max_subs_update, y_min_subs_update, y_max_subs_update]
print(extent_subs_update)

[887960.0, 890777.0, 2754170.0, 2752380.0]


In [46]:
geotrans_subs = [x_min_subs_update, x_res, 0,y_max_subs_update, 0, y_res]
print(geotrans_subs)

[887960.0, 10.0, 0, 2752380.0, 0, -10.0]


In [47]:
img_array = dset.ReadAsArray()
img_array.shape
##以上是波段数，行数，列数


(6, 3261, 2980)

In [51]:
img_array_subs = img_array[:, \
                           row_start_subs_update:row_start_subs_update+y_size_subs_update, \
                            col_start_subs_update:col_start_subs_update+x_size_subs_update]
print(img_array_subs.shape)

(6, 0, 282)


In [52]:
path_subs = 'E:\开源\ynu_cj.tif'
driver = gdal.GetDriverByName('GTiff')
dset_subs = driver.Create(path_subs,
                          xsize=x_size_subs_update,
                          ysize=y_size_subs_update,
                          bands=dset.RasterCount,
                          eType=gdal.GDT_Int16)
dset_subs.SetGeoTransform(geotrans_subs)
dset_subs.SetProjection(dset.Getprojection())
for i in range(dset.RasterCount):
    outband = dset_subs.GetRasterBand(i+1)
    outband.WriteArray(img_array_subs[i])
dset_subs= None

AttributeError: 'NoneType' object has no attribute 'SetGeoTransform'